In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.svm import LinearSVC
from ml_lib import LinearSVC as MyLinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [4]:
X, y = make_moons(n_samples=100, noise=0.1, random_state=42)

csvc = MyLinearSVC(C=1.0, max_iter=1000, learning_rate=0.1, tol=1e-1)
svc = LinearSVC(C=1.0, tol=1e-1, max_iter=1000)
scaler = StandardScaler()
X = scaler.fit_transform(X)

csvc.fit(X, y)
svc.fit(X, y)

y_pred = csvc.predict(X)
print("MyLinearSVC accuracy:", accuracy_score(y, y_pred))

y_pred = svc.predict(X)
print("LinearSVC accuracy:", accuracy_score(y, y_pred))

MyLinearSVC accuracy: 0.51
LinearSVC accuracy: 0.87
